In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from skimage.io import imshow
from pathlib import Path
import pandas as pd

In [5]:
dataset_root = Path('../MURA-v1.1')

In [6]:
list(dataset_root.iterdir())

[PosixPath('../MURA-v1.1/valid'),
 PosixPath('../MURA-v1.1/train_labeled_studies.csv'),
 PosixPath('../MURA-v1.1/valid_image_paths.csv'),
 PosixPath('../MURA-v1.1/train'),
 PosixPath('../MURA-v1.1/valid_labeled_studies.csv'),
 PosixPath('../MURA-v1.1/train_image_paths.csv')]

In [7]:
df = pd.read_csv(dataset_root/'train_image_paths.csv', header=None, names=['filename'])
df.head()

,filename
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [8]:
df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
df.head()

,filename,class
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive


In [9]:
def generate_df(dataset_root, csv_name):
    df = pd.read_csv(dataset_root/csv_name, header=None, names=['filename'])
    df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
    return df

In [10]:
list(dataset_root.parent.iterdir())

[PosixPath('../Notebooks 2019-20'),
 PosixPath('../.DS_Store'),
 PosixPath('../MURA-v1.1'),
 PosixPath('../DeepLearning2.ipynb'),
 PosixPath('../exec2'),
 PosixPath('../fashion-mnist-image-classification'),
 PosixPath('../.ipynb_checkpoints'),
 PosixPath('../Lectures 2019-20')]

In [11]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255)
train_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'train_image_paths.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')
valid_gen = datagen.flow_from_dataframe(generate_df(dataset_root, 'valid_image_paths.csv'),
                                        directory=dataset_root.parent,
                                        target_size=(224, 224),
                                        class_mode='binary')

Found 36808 images belonging to 2 classes.
Found 3197 images belonging to 2 classes.


In [12]:
densenet = tf.keras.applications.DenseNet169(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.
51879936/51877672 [==============================] - 26s 0us/step


In [13]:
densenet.trainable = False

In [14]:
densenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [15]:
model = tf.keras.models.Sequential([
    densenet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet169 (Model)          (None, 7, 7, 1664)        12642880  
_________________________________________________________________
flatten (Flatten)            (None, 81536)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 81537     
Total params: 12,724,417
Trainable params: 81,537
Non-trainable params: 12,642,880
_________________________________________________________________


In [17]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [18]:
model.fit_generator(train_gen, epochs=5, validation_data=valid_gen, use_multiprocessing=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1151/1151 [==============================] - 11028s 10s/step - loss: 1.6111 - acc: 0.6736 - val_loss: 2.1063 - val_acc: 0.6872
Epoch 2/5
1151/1151 [==============================] - 13763s 12s/step - loss: 1.6047 - acc: 0.7301 - val_loss: 1.7740 - val_acc: 0.7129
Epoch 3/5
1151/1151 [==============================] - 12399s 11s/step - loss: 1.6891 - acc: 0.7392 - val_loss: 2.0944 - val_acc: 0.6788
Epoch 4/5
1151/1151 [==============================] - 8299s 7s/step - loss: 1.6443 - acc: 0.7525 - val_loss: 2.1927 - val_acc: 0.7063
Epoch 5/5
1151/1151 [==============================] - 8052s 7s/step - loss: 1.7111 - acc: 0.7626 - val_loss: 2.7245 - val_acc: 0.6691


In [20]:
model.metrics_names

['loss', 'acc']

In [19]:
model.evaluate_generator(generator=valid_gen)

[2.723571994304657, 0.66906476]